<a href="https://colab.research.google.com/github/tedyadika/Azureml/blob/main/Data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install azureml.core
!pip install numpy == 1.20
!pip install pycaret
!pip install azureml-dataset-runtime --upgrade
!pip install lazypredict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.4/141.4 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.9/965.9 KB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

### Import Azure ML SDK modules

In [3]:
from azureml.core import Workspace, Datastore,Dataset,Experiment,Run

### Import python modules

In [4]:
%matplotlib inline
import datetime
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date,timedelta
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [5]:
data_read ='country_sdk'
filter_column ='' #'ORDTYP_map'
filter_value = ''#"ECOMM"
col_lag_list = ['TOTAL_ORDNUM_LS','DW_SUM_ORDQTY_AD','DW_QTY_REC_LS','SUM_QTY_PIECE_PICKS','TOTAL_ORDNUM_AD']
split_column= 'COUNTRY_NAME'
target_column = 'SUM_QTY_PIECE_PICKS'
cols_drop = ['SUM_QTY_PIECE_PICKS','NBR_ORDERS','SUM_QTY_TOTAL_PICKS','DW_SUM_ORDQTY_AD','DW_QTY_REC_LS','TOTAL_ORDNUM_LS','TOTAL_ORDNUM_AD']

### Set up and data import 

In [6]:
ws = Workspace.get(name= 'dscglobalceedamlt',
                    subscription_id= '8b20e38c-72fb-4030-a702-128e713479ca',
                    resource_group= 'DSCGlobalCEEDTANLTst')

az_store= Datastore.get(ws,'dscglobalceedamlsat_datastore')
az_dataset= Dataset.get_by_name(ws,data_read)
az_default_store = ws.get_default_datastore()

Performing interactive authentication. Please follow the instructions on the terminal.


Interactive authentication successfully completed.


### set up experiment

In [7]:
experiment_run = Run.get_context()
dataset1 =az_dataset.to_pandas_dataframe()

Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'


In [8]:
df = dataset1.copy()
if filter_column and filter_value !=None:
    df = df.query(f'{filter_column}  == "{filter_value}"')
elif filter_column and filter_value == None:
    output_log = 'NO_FILTER APPLIED'
    experiment_run.log(output_log)

In [9]:
nulldf = df.isnull().sum()
for col in df.columns:
    experiment_run.log(col, nulldf[col])

Attempted to log scalar metric DATE:
3
Attempted to log scalar metric ORDTYP_map:
0
Attempted to log scalar metric COUNTRY_NAME:
0
Attempted to log scalar metric NBR_ORDERS:
0
Attempted to log scalar metric DW_SUM_ORDQTY_AD:
16348
Attempted to log scalar metric DW_QTY_REC_LS:
4520
Attempted to log scalar metric TOTAL_ORDNUM_LS:
4122
Attempted to log scalar metric TOTAL_ORDNUM_AD:
16348
Attempted to log scalar metric SUM_QTY_TOTAL_PICKS:
0
Attempted to log scalar metric SUM_QTY_PIECE_PICKS:
0


### spliting our data by country 

In [10]:
# Initialize an empty list
df_list = []
# Split the dataframe into separate datasets based on the values in the 'country' column
for name, group in df.groupby(split_column):
  df_list.append(group)

### Adding importang parameters to our data

In [11]:
def add_values(data):
    #data = df.copy()
    data['DATE'] = pd.to_datetime(data['DATE'],format ='%Y-%m-%d', errors='coerce')
    data.sort_values('DATE', inplace=True)
    #data = data.resample('W', on='DATE').sum()
    #data.reset_index(inplace = True)
    data['logarithm_base2'] = np.log10(data[target_column])
    data['MA14'] = data[target_column].shift(14).rolling(7).mean()
    data['MA7'] = data[target_column].shift(7).rolling(14).mean()
    data['Month'] = [i.month for i in data['DATE']]
    data['Year'] = [i.year for i in data['DATE']]
    data['WEEK_YEAR'] = data['DATE'].dt.isocalendar().week 
    data['day_of_week'] = data['DATE'].dt.dayofweek
    data['DAY_YEAR'] = data['DATE'].dt.dayofyear
    data['DATE_N'] = data['DATE'].apply(lambda x:x.toordinal())
    data["WEEK_YEAR"] = data["WEEK_YEAR"].astype('int64').astype('int64')
    return(data)



In [12]:
new_col = ['logarithm_base2']
#col_lag_list = col_lag_list + new_col
col_lag_list

['TOTAL_ORDNUM_LS',
 'DW_SUM_ORDQTY_AD',
 'DW_QTY_REC_LS',
 'SUM_QTY_PIECE_PICKS',
 'TOTAL_ORDNUM_AD']

### Adding Lags

In [13]:
lag = [7,14,21,28,35]
def add_lags(df_list,column_list,lag_list):
  for i in range(len(df_list)):
      df = df_list[i]#for df in df_list:
      df = df.dropna(subset=['DATE'])
      add_values(df)
      for col in col_lag_list:
        print(col)
        for l in lag:
          new_col_name = col + '_lag_' + str(l)
          new_col_name2 = col + '_mean_' + str(l)
          df_list[i][new_col_name] = df_list[i][col].shift(l)
          df_list[i][new_col_name2] = df_list[i][col].shift(l).rolling(l).mean()
          df_list[i] = df_list[i].fillna(df.mean())


add_lags(df_list,col_lag_list,lag)


/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-13-923892e5ea6d>:14: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df_list[i] = df_list[i].fillna(df.mean())
<ipython-input-13-923892e5ea6d>:14: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_list[i] = df_list[i].fillna(df.mean())


TOTAL_ORDNUM_LS
DW_SUM_ORDQTY_AD
DW_QTY_REC_LS
SUM_QTY_PIECE_PICKS
TOTAL_ORDNUM_AD
TOTAL_ORDNUM_LS
DW_SUM_ORDQTY_AD


/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-13-923892e5ea6d>:14: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df_list[i] = df_list[i].fillna(df.mean())
<ipython-input-13-923892e5ea6d>:14: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_list[i] = df_list[i].fillna(df.mean())
/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


DW_QTY_REC_LS
SUM_QTY_PIECE_PICKS
TOTAL_ORDNUM_AD
TOTAL_ORDNUM_LS
DW_SUM_ORDQTY_AD


<ipython-input-13-923892e5ea6d>:14: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df_list[i] = df_list[i].fillna(df.mean())
<ipython-input-13-923892e5ea6d>:14: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_list[i] = df_list[i].fillna(df.mean())


...

TOTAL_ORDNUM_LS
DW_SUM_ORDQTY_AD
DW_QTY_REC_LS
SUM_QTY_PIECE_PICKS
TOTAL_ORDNUM_AD


<ipython-input-11-cfc4f711f5f5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['MA14'] = data[target_column].shift(14).rolling(7).mean()
<ipython-input-11-cfc4f711f5f5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['MA7'] = data[target_column].shift(7).rolling(14).mean()
<ipython-input-11-cfc4f711f5f5>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

TOTAL_ORDNUM_LS
DW_SUM_ORDQTY_AD
DW_QTY_REC_LS
SUM_QTY_PIECE_PICKS
TOTAL_ORDNUM_AD


/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-13-923892e5ea6d>:14: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df_list[i] = df_list[i].fillna(df.mean())
<ipython-input-13-923892e5ea6d>:14: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_list[i] = df_list[i].fillna(df.mean())
/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-13-923892e5ea6d>:14: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 

TOTAL_ORDNUM_LS
DW_SUM_ORDQTY_AD
DW_QTY_REC_LS
SUM_QTY_PIECE_PICKS
TOTAL_ORDNUM_AD
TOTAL_ORDNUM_LS
DW_SUM_ORDQTY_AD
DW_QTY_REC_LS
SUM_QTY_PIECE_PICKS
TOTAL_ORDNUM_AD


### Resampling data and droping columns 
in this function we resample our data on weekly basis  add date values, drop unwanted columns ,
drop columns with very low correlation and fill nan with mean 

In [17]:
for i in range(len(df_list)):
    ml_data1 = df_list[i].copy()
    name = ml_data1[split_column].unique()
    ml_data = ml_data1.resample('W', on='DATE').sum()
    ml_data.reset_index(inplace = True)
    ml_data = add_values(ml_data)
    ml_data = ml_data.drop(cols_drop, axis=1)
    ml_data.reset_index(inplace = True)
    ml_data[split_column] = name[0]

    corr_matrix = ml_data.corr()
    column_of_interest = 'logarithm_base2'
    corr = corr_matrix[column_of_interest].loc[corr_matrix[column_of_interest] != 1]
    threshold = 0.3
    columns_to_drop = corr[corr < threshold].index.tolist()
    ml_data = ml_data.drop(columns_to_drop, axis=1)
    ml_data=ml_data.mask(ml_data==0).fillna(ml_data.mean())

    try:
        Dataset.Tabular.register_pandas_dataframe(dataframe = ml_data,
                                               target = az_store,
                                               name = name[0]+'_DATA_FOR_ML',
                                                description=name[0]+'READY FOR ML')
    except Exception as ex:
        print('Error register_pandas_dataframe: ', ex)


TypeError: ignored

### complete the experiment

In [ ]:
experiment_run.complete()